# Imports and types

In [21]:
import random
import time
import numpy as np
import pandas as pd
import psutil
import os

In [22]:
def calcular_aptidao(individuo):
    return sum(individuo)

In [23]:
def mutacao(solucao):
    nova = solucao[:]
    a, b = random.sample(range(len(nova)), 2)
    nova[a], nova[b] = nova[b], nova[a]
    return nova

In [24]:
def gerar_vizinho(solucao):
    return mutacao(solucao)

In [25]:
def gerar_array_replicavel(seed: int, tamanho: int) -> list[int]:
    random.seed(seed)
    vetor = list(range(tamanho))  
    random.shuffle(vetor)
    return vetor

In [26]:
def calcular_aptidao_qap(solucao, matriz_fluxo, matriz_distancia):
    n = len(solucao)
    custo = 0
    for i in range(n):
        for j in range(n):
            custo += matriz_fluxo[i][j] * matriz_distancia[solucao[i]][solucao[j]]
    return custo

In [27]:
def VND(solucao, matriz_fluxo, matriz_distancia, kmax=5):
    melhor = solucao[:]
    k = 1
    while k <= kmax:
        vizinho = gerar_vizinho(melhor)
        if calcular_aptidao_qap(vizinho, matriz_fluxo, matriz_distancia) < calcular_aptidao_qap(melhor, matriz_fluxo, matriz_distancia):
            melhor = vizinho
            k = 1
        else:
            k += 1
    return melhor

In [28]:
def ES_VND(mu, lambd, tempo_max, taxa_mutacao, taxa_busca_local, iter_sem_melhora_max,
           solucao_inicial, matriz_fluxo, matriz_distancia):

    P = [mutacao(solucao_inicial) for _ in range(lambd)]
    melhor = None
    melhor_aptidao = float('inf')
    sem_melhora = 0

    inicio = time.time()

    while (time.time() - inicio) < tempo_max and sem_melhora < iter_sem_melhora_max:
        aptidoes = [calcular_aptidao_qap(ind, matriz_fluxo, matriz_distancia) for ind in P]

        for ind, apt in zip(P, aptidoes):
            if apt < melhor_aptidao:
                melhor_aptidao = apt
                melhor = ind[:]
                sem_melhora = 0
        sem_melhora += 1

        melhores_indices = sorted(range(len(P)), key=lambda i: aptidoes[i])[:mu]
        Q = [P[i][:] for i in melhores_indices]

        nova_geracao = Q[:]
        for q in Q:
            for _ in range(lambd // mu):
                individuo = q[:]
                if random.random() < taxa_mutacao:
                    individuo = mutacao(individuo)
                if random.random() < taxa_busca_local:
                    individuo = VND(individuo, matriz_fluxo, matriz_distancia)
                nova_geracao.append(individuo)

        P = nova_geracao

    return melhor, melhor_aptidao

In [29]:
def ler_qap_com_n(caminho: str):
    with open(caminho, "r") as f:
        dados = list(map(int, f.read().split()))
        
    n = dados[0]
    valores = dados[1:]  

    total_esperado = 2 * n * n
    if len(valores) != total_esperado:
        raise ValueError(f"Esperado {total_esperado} valores, mas encontrado {len(valores)}.")

    flow_flat = valores[:n * n]
    dist_flat = valores[n * n:]

    flow_df = pd.DataFrame([flow_flat[i * n:(i + 1) * n] for i in range(n)])
    dist_df = pd.DataFrame([dist_flat[i * n:(i + 1) * n] for i in range(n)])

    return n, flow_df, dist_df

In [ ]:
if __name__ == "__main__":
    arquivo = "Lipa80b.txt"
    nome_instancia = os.path.splitext(os.path.basename(arquivo))[0]

    n, flow_df, dist_df = ler_qap_com_n(arquivo)
    flow = flow_df.values.tolist()
    dist = dist_df.values.tolist()
    matriz_fluxo = np.array(flow)
    matriz_distancia = np.array(dist)

    resultados = []

    for seed in range(42, 52):  
        random.seed(seed)

        # Monitor de tempo e memória
        process = psutil.Process(os.getpid())
        memoria_inicial = process.memory_info().rss / (1024 * 1024)  
        tempo_inicio = time.time()

        solucao_inicial = gerar_array_replicavel(seed=seed, tamanho=n)

        parametros = {
            "mu": random.randint(5, 10),
            "lambd": random.randint(30, 100),
            "tempo_max": random.randint(3, 10) * 60,
            "taxa_mutacao": random.uniform(0.4, 0.7),
            "taxa_busca_local": random.uniform(0.4, 0.7),
            "iter_sem_melhora_max": random.randint(5, 10),
            "solucao_inicial": solucao_inicial,
            "matriz_fluxo": matriz_fluxo,
            "matriz_distancia": matriz_distancia
        }

        melhor_solucao, melhor_valor = ES_VND(**parametros)

        tempo_fim = time.time()
        memoria_final = process.memory_info().rss / (1024 * 1024)  # MB

        resultados.append({
            "instancia": nome_instancia,
            "seed": seed,
            "melhor_solucao": melhor_solucao,
            "custo": melhor_valor,
            "tempo_execucao_segundos": round(tempo_fim - tempo_inicio, 2),
            "memoria_usada_MB": round(memoria_final - memoria_inicial, 2)
        })

        print(f"Seed {seed} finalizada. Custo: {melhor_valor}")

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(f"resultados_{nome_instancia}.csv", index=False)

    print(f"\nResultados salvos em resultados_{nome_instancia}.csv")

Seed 42 finalizada. Custo: 9520085
Seed 43 finalizada. Custo: 9508540
Seed 44 finalizada. Custo: 9525871
Seed 45 finalizada. Custo: 9512868
Seed 46 finalizada. Custo: 9566457
Seed 47 finalizada. Custo: 9525759
Seed 48 finalizada. Custo: 9541981
Seed 49 finalizada. Custo: 9511561
Seed 50 finalizada. Custo: 9567223
